In [1]:
import sys
sys.path.append("../..")
from thesis.utils.utils import setup_ray, save, load, Experiment
path = "D:/Master/Masterarbeit/thesis"
setup_ray(path = path, unidirectional = False, seed=69)

2023-03-22 06:20:28,475	INFO worker.py:1528 -- Started a local Ray instance.


In [2]:
env_args = dict(
    fleetsize = 4,
    max_fleetsize = 20,    
    pseudo_routing = False,
    pseudo_dispatcher = False,
    pseudo_dispatcher_clever = False,
    #pseudo_dispatcher_distance = 0.3,
    routing_agent_death= False,
    death_on_target = False,
    transform_dispatching_partobs = True,
    direction_reward = 0,
    sim_config = dict(
        dispatch = True,
        routing_ma = True,
        dispatching_ma = True,
        reward_reached_target = 0,
        #reward_reached_target_by_time = True, 
        reward_wrong_target = 0,
        reward_removed_for_block = 0, 
        reward_target_distance = 0,
        reward_invalid= 0,
        reward_duration = 0,
        block_timeout = 120,
        station_separate = True,
        reward_accepted_in_station = 2,
        reward_declined_in_station = -1,
        #reward_part_completed = 5,
        #reward_geo_operation=1,
        #reward_rework_operation=1,
        #reward_respot_operation=1,
        reward_reduce = -0.02,
        routing_interval = 2,
        dispatching_interval=30,
        io_quote = 0.9  ,
        availability = 0.9,
        mttr = 5,
    )
)

In [3]:
agv_model = dict(
    model = dict(
        custom_model = "gnn_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=16,
            with_action_mask=False,
            with_agvs=True,
            with_stations = False,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
            env_type = "matrix",
            n_convolutions = 2
        )
    )
)
dispatcher_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=32,
            with_action_mask=True,
            with_agvs=True,
            with_stations = True,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
        )
    )
)

In [4]:
exp = Experiment("matrix_dispatching")
for seed in [42]:
    exp.experiment(
        path = path,
        env_args = env_args, 
        agv_model = agv_model,
        dispatcher_model=dispatcher_model,
        run_name="09_train_on_RL", 
        env = "matrix",
        algo = "ppo",
        n_intervals =0,
        train_agv = False,
        backup_interval=40,
        batch_size=500, #apex + gnn: 50
        seed = seed,
        algo_params = {"gamma":0.9,},#"grad_clip": 1,  "exploration_config":{"warmup_timesteps": 0,"epsilon_timesteps": 200000,"final_epsilon": 0.02,"initial_epsilon": 1.0,"type": "EpsilonGreedy"}},# "exploration_config": {"type": "Curiosity", "sub_exploration": {"type": "StochasticSampling"}, "eta": 0.1}},
        lr = 1e-4,
        #load_agv="../../models/matrix_routing/04_rew_dir_-0_1_8_20_2023-02-28_23-41-18/checkpoint_000400",
        
    )

2023-03-22 06:20:39,216	WARNING ppo.py:351 -- `train_batch_size` (500) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=8 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 63.
2023-03-22 06:20:39,246	INFO algorithm.py:457 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_26434_j0650m3n)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_26449_u5tn5vel)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary dir

In [5]:
load(exp.trainer, "agv", "../../models/trained_routing")

In [6]:
exp.keep_training(3)